In [13]:
from prometheus_client import Counter
from sage.all import *
import base64
from Crypto.Cipher import AES
from hashlib import sha256
from Crypto.Util.number import long_to_bytes
from Crypto.Util.Padding import unpad
import base64

p = 309465533233587653298203953963739275397
shares = [
    (1, 201522632269176227792529259658745111658),
    (2, 224770860677244800791621415404633393675),
    (3, 135324715440419453670163953081590813641),
    (4, 120473893289346312314482711088749854173),
    (6, 279670438188058666671026211629408563184), 
]

F = GF(p)                 # trường hữu hạn mod p
R.<x> = PolynomialRing(F) # đa thức trên F

points = [(F(xi), F(yi)) for xi, yi in shares]
f = R.lagrange_polynomial(points)
coeffs = f.list()

secret = coeffs[0] 

print("Lagrange polynomial f(x):", f)
secret_int = int(secret)               
secret_bytes = long_to_bytes(secret_int)  
print("Secret bytes length:", len(secret_bytes))

key = secret_bytes.hex()
key = bytes.fromhex(key)
print("Derived key (hex):", key)

input_path = "./final/vault.enc"
with open(input_path, "rb") as f:
    b64_data = f.read()
    
data = base64.b64decode(b64_data)
print("Total decoded length:", len(data))

header = data
print("Header (16 bytes, skipped):", header.hex())

iv = data[:16]
ciphertext = data[16:]

print("IV (16 bytes):", iv.hex())
print("Ciphertext length:", len(ciphertext))

print("Ciphertext preview (first 200 bytes):")
print(ciphertext.hex())


cipher = AES.new(key, AES.MODE_CFB, iv)
plaintext = cipher.decrypt(ciphertext)
print("Decrypted plaintext length:", plaintext)

cipher_cbc = AES.new(key, AES.MODE_CBC, iv)
plaintext_cbc = cipher_cbc.decrypt(ciphertext)
print("CBC decrypted plaintext length:", plaintext_cbc)

# --- CFB Mode ---
cipher_cfb = AES.new(key, AES.MODE_CFB, iv)
plaintext_cfb = cipher_cfb.decrypt(ciphertext)
print("CFB decrypted plaintext length:", plaintext_cfb)

# --- OFB Mode ---
cipher_ofb = AES.new(key, AES.MODE_OFB, iv)
plaintext_ofb = cipher_ofb.decrypt(ciphertext)
print("OFB decrypted plaintext length:", plaintext_ofb)

# # --- CTR Mode ---
# # CTR dùng counter, tạo từ IV
# ctr = Counter.new(128, initial_value=int.from_bytes(iv, byteorder='big'))
# cipher_ctr = AES.new(key, AES.MODE_CTR, counter=ctr)
# plaintext_ctr = cipher_ctr.decrypt(ciphertext)
# print("CTR decrypted plaintext length:", plaintext_ctr)

# --- GCM Mode ---
# GCM thường có nonce 12 bytes, nhưng bạn có thể dùng 16 bytes
cipher_gcm = AES.new(key, AES.MODE_GCM, nonce=iv)
try:
    # Nếu có tag, dùng cipher_gcm.decrypt_and_verify(ciphertext, tag)
    plaintext_gcm = cipher_gcm.decrypt(ciphertext)
    print("GCM decrypted plaintext length:", plaintext_gcm)
except ValueError as e:
    print("GCM decryption failed:", e)

Lagrange polynomial f(x): 217828790202859527943075149706537689961*x^4 + 70763368933890590187431146074609987838*x^3 + 107930742557680539553236674147348263163*x^2 + 269198030425127050056092220675858083805*x + 154732766616793826649101976981869637685
Secret bytes length: 16
Derived key (hex): b'thresh0ld_k3y_05'
Total decoded length: 64
Header (16 bytes, skipped): 66697865645f69765f313662797465735bc5b232dd555605580bb70cb6bbb36453500d3b1a6ee5d0d67187d81b72ade2d6198cc121563d279824ef728ff14c18
IV (16 bytes): 66697865645f69765f31366279746573
Ciphertext length: 48
Ciphertext preview (first 200 bytes):
5bc5b232dd555605580bb70cb6bbb36453500d3b1a6ee5d0d67187d81b72ade2d6198cc121563d279824ef728ff14c18
Decrypted plaintext length: b"4\xcf`\xf4_\x9b,\x82\xe3\xa6\xbaCJ>\x938\xf1\xba\r\xb5\xa7\x8f\xefo'\x17i\xedZ\xb59x5}15\xef\xa6-\x95\xa6V<\x1bd\xcf\xa8\xde"
CBC decrypted plaintext length: b'CSCV2025{Thresh0ld_c1pher_2025@?!}\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
CFB decrypted plainte

In [6]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
print(bytes_to_long('Hello CTF'.encode('utf-8')))
print(bytes_to_long('print(Hello CTF)'.encode('utf-8')))
print(bytes_to_long('AMOARHODHAEASZOMM'.encode('utf-8')))

print(len(long_to_bytes(66799091487232316287802301847294550361)))

1335473908826941641798
149467595773838768162731789920272401961
22221115921863975210544525821466526043469
16


In [7]:
#!/usr/bin/env python3

# Prime từ file prime
p = 309465533233587653298203953963739275397

# Các shares đã thu thập được (x, y)
shares = [
    (1, 201522632269176227792529259658745111658),
    (2, 224770860677244800791621415404633393675),
    (3, 135324715440419453670163953081590813641),
    (4, 120473893289346312314482711088749854173),
    (6, 279670438188058666671026211629408563184),
]

def mod_inverse(a, m):
    """Tính modular inverse của a mod m"""
    def extended_gcd(a, b):
        if a == 0:
            return b, 0, 1
        gcd, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y
    
    _, x, _ = extended_gcd(a % m, m)
    return (x % m + m) % m

def lagrange_interpolation(shares, x, p):
    """
    Nội suy Lagrange tại điểm x mod p
    shares: list of (xi, yi) tuples
    """
    result = 0
    
    for i, (xi, yi) in enumerate(shares):
        # Tính Lagrange basis polynomial L_i(x)
        numerator = 1
        denominator = 1
        
        for j, (xj, _) in enumerate(shares):
            if i != j:
                numerator = (numerator * (x - xj)) % p
                denominator = (denominator * (xi - xj)) % p
        
        # L_i(x) = numerator / denominator mod p
        lagrange_basis = (numerator * mod_inverse(denominator, p)) % p
        
        # Cộng vào kết quả: f(x) = sum(yi * L_i(x))
        result = (result + yi * lagrange_basis) % p
    
    return result

# Tính secret (f(0))
secret = lagrange_interpolation(shares, 0, p)
print(f"Secret f(0) = {secret}")

# Chuyển secret thành AES key (hex)
secret_hex = hex(secret)[2:]
if len(secret_hex) % 2:
    secret_hex = '0' + secret_hex
print(f"Secret (hex) = {secret_hex}")

# Chuyển hex sang ASCII
try:
    secret_ascii = bytes.fromhex(secret_hex).decode('ascii')
    print(f"AES Key (ASCII) = {secret_ascii}")
except:
    print(f"Cannot decode as ASCII")

# Tính share5 và share7 (nếu cần)
share5_y = lagrange_interpolation(shares, 5, p)
share7_y = lagrange_interpolation(shares, 7, p)
print(f"\nShare 5: f(5) = {share5_y}")
print(f"Share 7: f(7) = {share7_y}")

# Decrypt vault.enc
print("\n" + "="*60)
print("Decrypting vault.enc...")
print("="*60)

try:
    from Crypto.Cipher import AES
    from Crypto.Util.Padding import unpad
    import base64
    
    # Đọc vault.enc
    with open('vault.enc', 'rb') as f:
        vault_data = f.read()
    
    # Decode base64
    decoded = base64.b64decode(vault_data)
    
    # Tách IV và ciphertext
    iv = decoded[:16]
    ciphertext = decoded[16:]
    
    print(f"IV (hex): {iv.hex()}")
    print(f"Ciphertext length: {len(ciphertext)} bytes")
    
    # AES key (thresh0ld_k3y_05)
    key = b'thresh0ld_k3y_05'
    
    # Decrypt
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)
    
    # Unpad
    try:
        plaintext = unpad(plaintext, AES.block_size)
    except:
        # Remove zero padding manually
        plaintext = plaintext.rstrip(b'\x00')
    
    print(f"\n{'='*60}")
    print("DECRYPTED PLAINTEXT:")
    print('='*60)
    print(plaintext.decode('utf-8', errors='ignore'))
    print('='*60)
    
except ImportError:
    print("\nCần cài đặt pycryptodome: pip install pycryptodome")
except FileNotFoundError:
    print("\nKhông tìm thấy file vault.enc")
except Exception as e:
    print(f"\nLỗi khi decrypt: {e}")

Secret f(0) = 154732766616793826649101976981869637685
Secret (hex) = 746872657368306c645f6b33795f3035
AES Key (ASCII) = thresh0ld_k3y_05

Share 5: f(5) = 25019457618722413877035949434840599806
Share 7: f(7) = 44250631720153930381176600900379032678

Decrypting vault.enc...

Không tìm thấy file vault.enc
